# Parameters for simulations
This notebook allows to create a config.json file containing all the parameters for the simulation to be passed to the main script scripts/run_simulations.py

In [2]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

### SINGLE SIMULATION ###

In [ ]:
# System configuration: edit name of the dynamical system and its parameters
from dyn_net.utils.criticality import find_theta_c_from_degree_distribution

# Degree distribution used by the configuration model
N = 10_000
is_critical = False # if true \theta = \theta_c , if false \theta = 0.5* \theta_c

degree_distribution = {
    "name": "poisson",
    "params": {
        "lambda": 5,
    },
}


system = {
    "name": "double_well_network",
    "params": {
        "theta": None, # Will be changed below
        "alpha_rot": 0.0,
    },
}

# Network configuration: edit name, size = number of particles and parameters of the network
network = {
    "name": "configuration_model",
    "params": {
        "n": N,
        "degree_distribution": degree_distribution,
    },
}

# Noise configuration
sigma = 0.4
noise = {
    "name": "additive_gaussian",
    "params": {
        "sigma": sigma,
    },
}

# Integrator configuration
integrator = {
    "tmin": 0.0,
    "tmax": 5000.0 + 2*10**5,
    "dt": 0.01,
    "stats_every": 200, # every "stats_every" timesteps get the appropriate statistics and write them to file
    "state_every": 200, # every "state_every" timesteps write the state of the system to file
    "write_stats_at_start": True,
    "write_state_at_start": True,
}

# Initial condition configuration
initial_condition = {
    "type": "normal", # choose the type of initial condition + its parameters
    "std": 1.0,
}

config = {
    "system": system,
    "network": network,
    "noise": noise,
    "integrator": integrator,
    "initial_condition": initial_condition,
}


# Set the value of the interaction strength

theta_c = find_theta_c_from_degree_distribution(
    degree_distribution=degree_distribution,
    sigma = noise["params"]["sigma"],
    theta_bracket=(0,1))

if is_critical:
    system["params"]["theta"] = theta_c
    label = "critical"
else: 
    system["params"]["theta"] = 0.5*theta_c
    label = "far"


config = {
    "system": system,
    "network": network,
    "noise": noise,
    "integrator": integrator,
    "initial_condition": initial_condition,
}


# The json file will be saved at root_folder/path_folder/name_config_file
path_folder = f"configs/linear_response/poisson/unperturbed_runs/{label}/n{str(N)}/"
name_config_file = "config_double_well_configuration_model.json"

# Save it in the configs folder
repo_root = Path.cwd().parent
output_path = repo_root / path_folder / name_config_file
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(json.dumps(config, indent=2))
print(f"Wrote config to {output_path}")


### PHASE DIAGRAM SIMULATIONS ###

In [ ]:
# System configuration: edit name of the dynamical system and its parameters
from dyn_net.utils.criticality import find_theta_c_from_degree_distribution

# Degree distribution used by the configuration model
N = 1000
network_label = "power_law"

degree_distribution = {
    "name": "scale_free_cutoff",
    "params": {
        "alpha": 2.5,
        "k_min" : 2,
        "k_max" : int( np.floor( np.sqrt( (2.5 - 1) / (2.5 - 2) * 2 * N ) ) )
    },
}


system = {
    "name": "double_well_network",
    "params": {
        "theta": None, # Will be changed below
        "alpha_rot": 0.0,
    },
}

# Network configuration: edit name, size = number of particles and parameters of the network
network = {
    "name": "configuration_model",
    "params": {
        "n": N,
        "degree_distribution": degree_distribution,
    },
}

# Noise configuration
sigma = 0.4
noise = {
    "name": "additive_gaussian",
    "params": {
        "sigma": sigma,
    },
}

# Integrator configuration
integrator = {
    "tmin": 0.0,
    "tmax": 5000.0,
    "dt": 0.01,
    "stats_every": 100, # every "stats_every" timesteps get the appropriate statistics and write them to file
    "state_every": 5000, # every "state_every" timesteps write the state of the system to file
    "write_stats_at_start": True,
    "write_state_at_start": True,
}

# Initial condition configuration
initial_condition = {
    "type": "normal", # choose the type of initial condition + its parameters
    "std": 1.0,
}

config = {
    "system": system,
    "network": network,
    "noise": noise,
    "integrator": integrator,
    "initial_condition": initial_condition,
}


config = {
    "system": system,
    "network": network,
    "noise": noise,
    "integrator": integrator,
    "initial_condition": initial_condition,
}


# The json file will be saved at root_folder/path_folder/name_config_file
path_folder = f"configs/phase_diagram/{network_label}/n{str(N)}/"
name_config_file = "config_double_well_configuration_model.json"

# Save it in the configs folder
repo_root = Path.cwd().parent
output_path = repo_root / path_folder / name_config_file
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(json.dumps(config, indent=2))
print(f"Wrote config to {output_path}")


# Define the sweep values
theta_c = find_theta_c_from_degree_distribution(
    degree_distribution=degree_distribution,
    sigma=noise["params"]["sigma"],
    theta_bracket=(1e-6, 1.0),
)

theta_values = np.concatenate([
    np.linspace(0.5 * theta_c, theta_c, 5),
    np.linspace(theta_c, 2 * theta_c, 10)[1:],
])

# Build a table with dot-path columns that override the base config
records = []
for theta_idx, theta in enumerate(theta_values, start=1):
    run_id = f"theta_{theta_idx:02d}"
    records.append({
        "run_id": run_id,
        "system.params.theta": float(theta),
    })

df = pd.DataFrame(records)

# Output path for the sweep table
path_folder = "params"
name_sweep_table = f"{network_label}/sweep_theta.tsv"
sweep_path = repo_root / path_folder / name_sweep_table
sweep_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(sweep_path, sep="	", index=False)
print(f"Wrote sweep table to {sweep_path}")

Wrote config to /Users/niccolo/Desktop/Dynamics_Networks/configs/phase_diagram/power_law/n1000/config_double_well_configuration_model.json
Wrote sweep table to /Users/niccolo/Desktop/Dynamics_Networks/params/power_law/sweep_theta.tsv


# Linear Response

### Create a seeds table for unperturbed runs
This writes a TSV with one row per graph realization, containing a unique network seed.
The base config is reused; stochastic dynamics remain non-reproducible because run.seed is not set.

Stochastic dynamics use OS entropy because no run.seed is set.
States are saved for initial conditions in the perturbed experiments.


In [ ]:
n_graphs = 1000
path_folder = "params/linear_response"
name_seed_table = "poisson_seeds_n1000.tsv"
seeds_path = repo_root / path_folder / name_seed_table
seeds_path.parent.mkdir(parents=True, exist_ok=True)

rng = np.random.default_rng()
records = []
for i in range(1, n_graphs + 1):
    run_id = f"graph_{i:03d}"
    records.append({
        "run_id": run_id,
        "network.params.seed": int(rng.integers(0, 2**32 - 1)),
    })

df_seeds = pd.DataFrame(records)
df_seeds.to_csv(seeds_path, sep="	", index=False)
print(f"Wrote seeds table to {seeds_path}")


### Create a response config (same system, response integrator, perturbation)
This writes a separate config for response experiments, reusing the same system/network/noise parameters
but with response-specific integrator settings and a perturbation epsilon.


In [19]:
# Response integrator configuration
response_integrator = {
    "tmin": 0.0,
    "tmax": 500.0,
    "dt": 0.01,
    "stats_every": 1,
    "state_every": 10_000,
    "write_stats_at_start": True,
    "write_state_at_start": True,
}

# Perturbation configuration
perturbation = {
    "type": "constant",
    "epsilon": 0.1,
}

response_config = {
    "system": system,
    "network": network,
    "noise": noise,
    "integrator": response_integrator,
    "perturbation": perturbation,
}

repo_root = Path.cwd().parent
output_path = repo_root / "configs" / "linear_response" / "perturbed_runs" / "poisson"
name_config_file = "config_double_well_configuration_model_poisson_n5000_far_constant.json"
output_path = output_path / name_config_file
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(json.dumps(response_config, indent=2))
print(f"Wrote response config to {output_path}")


Wrote response config to /Users/niccolo/Desktop/Dynamics_Networks/configs/linear_response/perturbed_runs/poisson/config_double_well_configuration_model_poisson_n5000_far_constant.json
